## Storm surge and wave climate
Notebook environment to migrate netcdf files to CF compliant zarr

In [ ]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

### Set up modules and paths

In [6]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

In [12]:
# Project paths & files (manual input)
WP_dir = coclico_data_dir.joinpath("WP3")
data_dir = WP_dir.joinpath("data")
ds_dir = data_dir.joinpath("NetCDF")
ds_filename = ds_dir.joinpath("CTP_MarineClimatologies.nc")
ds_out_filename = ds_dir.joinpath("CTP_MarineClimatologies")

CF_dir = data_dir.joinpath(r"CF")

In [19]:
ds = xr.open_dataset(ds_filename)
# Add CRS
ds.attrs["crs"] = 3035
ds

<xarray.Dataset>
Dimensions:      (locs: 51010)
Coordinates:
    lon          (locs) float64 ...
    lat          (locs) float64 ...
Dimensions without coordinates: locs
Data variables:
    Hsmean       (locs) float64 ...
    SSp99        (locs) float64 ...
    tidal_range  (locs) float64 ...
Attributes:
    title:         EU Coastal Target Points - Coastal marine climatologies
    summary:       Location of Coastal Target Points and their respective mea...
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      Waves, Storm surge, tide, coast, EU scale
    date_created:  28nd of February 2024
    project:       CoCliCo - Coastal Climate Core Services
    Conventions:   CF-1.8
    crs:           3035

In [20]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_filename, working_dir=CF_dir)

In [21]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_filename, working_dir=CF_dir)

# NOTE: File is already CF-compliant :)

{'P:\\11207608-coclico\\FULLTRACK_DATA\\WP3\\data\\NetCDF\\CTP_MarineClimatologies.nc': {'warnings': '0', 'errors': '0'}}


In [22]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_filename), mode="w")